Ejercicio: bosques aleatorios y arquitectura modelo

En el ejercicio anterior, usamos árboles de decisión para predecir si un crimen se resolvería en San Francisco.

Recuerde que los árboles de decisión hicieron un trabajo razonable, pero tienden a sobreajustarse, lo que significa que los resultados se degradarían considerablemente al usar el conjunto de prueba o cualquier dato no visto.

Esta vez usaremos bosques aleatorios para abordar esa tendencia de sobreajuste.

También veremos cómo la arquitectura del modelo puede influir en su rendimiento.

Visualización y preparación de datos
Como de costumbre, echemos otro vistazo rápido al conjunto de datos del crimen, luego divídalo en conjuntos de entrenamiento y prueba:

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/san_fran_crime.csv
import numpy as np
from sklearn.model_selection import train_test_split
import graphing # custom graphing code. See our GitHub repo for details

# Import the data from the .csv file
dataset = pandas.read_csv('san_fran_crime.csv', delimiter="\t")

# Remember to one-hot encode our crime and PdDistrict variables 
categorical_features = ["Category", "PdDistrict"]
dataset = pandas.get_dummies(dataset, columns=categorical_features, drop_first=False)

# Split the dataset in an 90/10 train/test ratio. 
# Recall that our dataset is very large so we can afford to do this
# with only 10% entering the test set
train, test = train_test_split(dataset, test_size=0.1, random_state=2, shuffle=True)

# Let's have a look at the data and the relationship we are going to model
print(dataset.head())
print("train shape:", train.shape)
print("test shape:", test.shape)

¡Espero que esto te resulte familiar! Si no, salte hacia atrás y realice el ejercicio anterior sobre árboles de decisión.

Código de evaluación del modelo
Usaremos el mismo código de evaluación del modelo que usamos en el ejercicio anterior

In [ ]:
from sklearn.metrics import balanced_accuracy_score

# Make a utility method that we can re-use throughout this exercise
# To easily fit and test out model

features = [c for c in dataset.columns if c != "Resolution"]

def fit_and_test_model(model):
    '''
    Trains a model and tests it against both train and test sets
    '''  
    global features

    # Train the model
    model.fit(train[features], train.Resolution)

    # Assess its performance
    # -- Train
    predictions = model.predict(train[features])
    train_accuracy = balanced_accuracy_score(train.Resolution, predictions)

    # -- Test
    predictions = model.predict(test[features])
    test_accuracy = balanced_accuracy_score(test.Resolution, predictions)

    return train_accuracy, test_accuracy


print("Ready to go!")

Árbol de decisión
Entrenemos rápidamente un árbol de decisión razonablemente bien ajustado para recordarnos su desempeño:

In [ ]:
import sklearn.tree
# re-fit our last decision tree to print out its performance
model = sklearn.tree.DecisionTreeClassifier(random_state=1, max_depth=10) 

dt_train_accuracy, dt_test_accuracy = fit_and_test_model(model)

print("Decision Tree Performance:")
print("Train accuracy", dt_train_accuracy)
print("Test accuracy", dt_test_accuracy)

Bosque aleatorio

Un bosque aleatorio es una colección de árboles de decisión que trabajan juntos para calcular la etiqueta de una muestra.

Los árboles en un bosque aleatorio se entrenan de forma independiente, en diferentes particiones de datos y, por lo tanto, desarrollan diferentes sesgos, pero cuando se combinan, es menos probable que sobreajusten los datos.

Construyamos un bosque muy simple con dos árboles y los parámetros predeterminados:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest model with two trees
random_forest = RandomForestClassifier( n_estimators=2,
                                        random_state=2,
                                        verbose=False)

# Train and test the model
train_accuracy, test_accuracy = fit_and_test_model(random_forest)
print("Random Forest Performance:")
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

Nuestro bosque de dos árboles lo ha hecho peor que el árbol único en el conjunto de prueba, aunque ha hecho un mejor trabajo en el conjunto de trenes.

Hasta cierto punto, esto debería esperarse. Los bosques aleatorios suelen funcionar con muchos más árboles. El simple hecho de tener dos le permitió sobreajustar los datos de entrenamiento mucho mejor que el árbol de decisión original.

Alterar el número de árboles.
Luego, construyamos varios modelos de bosque, cada uno con una cantidad diferente de árboles, y veamos cómo funcionan:

In [ ]:
import graphing

# n_estimators states how many trees to put in the model
# We will make one model for every entry in this list
# and see how well each model performs 
n_estimators = [2, 5, 10, 20, 50]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for n_estimator in n_estimators:
    print("Preparing a model with", n_estimator, "trees...")

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=n_estimator, 
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    n_estimators,
                    label_x="Numer of trees (n_estimators)",
                    label_y="Accuracy",
                    title="Performance X number of trees", show=True)

Las métricas se ven muy bien para el conjunto de entrenamiento, pero no tanto para el conjunto de prueba. Más árboles tendían a ayudar a ambos, pero solo hasta cierto punto.

Podríamos haber esperado que la cantidad de árboles resolviera nuestro problema de sobreajuste, ¡pero este no fue el caso! Lo más probable es que el modelo sea simplemente demasiado complejo en relación con los datos, lo que le permite sobreajustarse al conjunto de entrenamiento.

Alteración del número mínimo de muestras para el parámetro de división
Recuerde que los árboles de decisión tienen un nodo raíz, nodos internos y nodos hoja, y que los dos primeros se pueden dividir en nodos más nuevos con subconjuntos de datos.

Si dejamos que nuestro modelo se divida y creemos demasiados nodos, puede volverse cada vez más complejo y comenzar a sobreajustarse.

Una forma de limitar esa complejidad es decirle al modelo que cada nodo debe tener al menos una cierta cantidad de muestras; de lo contrario, no se puede dividir en subnodos.

En otras palabras, podemos establecer el parámetro min_samples_split del modelo en la menor cantidad de muestras necesarias para que un nodo se pueda dividir.

Nuestro valor predeterminado para min_samples_split es solo 2, por lo que los modelos rápidamente se volverán demasiado complejos si ese parámetro no se modifica.

Ahora usaremos el modelo de mejor rendimiento anterior, luego lo probaremos con diferentes valores de min_samples_split y compararemos los resultados:

In [ ]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:1000] # limit to 1000 samples

min_samples_split = [2, 10, 20, 50, 100, 500]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for min_samples in min_samples_split:
    print("Preparing a model with min_samples_split = ", min_samples)

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=20,
                                min_samples_split=min_samples,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    min_samples_split,
                    label_x="Minimum samples split (min_samples_split)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Rol back the trainset to the full set
train = full_trainset

Como puede ver arriba, las pequeñas restricciones en la complejidad del modelo, al limitar su capacidad para dividir nodos, reducen la brecha entre el entrenamiento y el rendimiento de la prueba. Si esto es sutil, lo hace sin dañar en absoluto el rendimiento de la prueba.

Al limitar la complejidad del modelo, abordamos el sobreajuste, mejorando su capacidad para generalizar y hacer predicciones precisas sobre datos no vistos.

Tenga en cuenta que usar min_samples_split=20 nos dio el mejor resultado para el conjunto de prueba y que los valores más altos empeoraron los resultados.

Alterar la profundidad del modelo
Un método relacionado para limitar los árboles es restringir max_depth. Esto es equivalente a max_ depth que usamos para nuestro árbol de decisiones, anteriormente. Su valor predeterminado es Ninguno, lo que significa que los nodos se pueden expandir hasta que todas las hojas sean puras (todas las muestras tienen la misma etiqueta) o tienen menos muestras que el valor establecido para min_samples_split.

Si max_ depth o min_samples_split son más apropiados depende de la naturaleza de su conjunto de datos, incluido su tamaño. Por lo general, necesitamos experimentar para encontrar la mejor configuración. Investiguemos max_ depth como si solo tuviéramos 500 muestras de delitos disponibles para nuestro conjunto de entrenamiento.

In [ ]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:500] # limit to 500 samples

max_depths = [2, 4, 6, 8, 10, 15, 20, 50, 100]

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for max_depth in max_depths:
    print("Preparing a model with max_depth = ", max_depth)

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=20,
                                max_depth=max_depth,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies),
                    max_depths,
                    label_x="Maximum depth (max_depths)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Rol back the trainset to the full set
train = full_trainset

El gráfico anterior nos dice que nuestro modelo en realidad se beneficia de un valor más alto para max_depth, hasta el límite de 15.

El aumento de la profundidad más allá de este punto comienza a dañar el rendimiento de la prueba, ya que restringe demasiado el modelo para que pueda generalizarse.

Como de costumbre, es importante evaluar diferentes valores al establecer los parámetros del modelo y definir su arquitectura.

Un modelo optimizado
La optimización adecuada de un modelo en un conjunto de datos tan grande puede llevar muchas horas, más de las que necesita para comprometerse con este ejercicio solo para aprender. Si desea ejecutar un modelo que ya se ha optimizado para el conjunto de datos completo, puede ejecutar el código a continuación y comparar su rendimiento con todo lo que hemos visto hasta ahora.

Esto es opcional; solo tenga en cuenta que el modelo puede tardar entre 1 y 2 minutos en entrenarse debido a su tamaño y al gran volumen de datos.

In [ ]:
# Prepare the model 
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=128,
                            max_features=25,
                            min_samples_split=2,
                            random_state=2, 
                            verbose=False)

# Train and test the result
print("Training model. This may take 1 - 2 minutes")
train_accuracy, test_accuracy = fit_and_test_model(rf)

# Print out results, compared to the decision tree
data = {"Model": ["Decision tree","Final random forest"],
        "Train sensitivity": [dt_train_accuracy, train_accuracy],
        "Test sensitivity": [dt_test_accuracy, test_accuracy]
        }

pandas.DataFrame(data, columns = ["Model", "Train sensitivity", "Test sensitivity"])

Como puede ver, el ajuste fino de los parámetros del modelo resultó en una mejora significativa en los resultados del conjunto de prueba.

Resumen
En este ejercicio cubrimos los siguientes temas:

Modelos de bosques aleatorios y en qué se diferencian de los árboles de decisión<br>
Cómo podemos cambiar la arquitectura de un modelo estableciendo diferentes parámetros y cambiando sus valores<br>
La importancia de probar varias combinaciones de parámetros y evaluar estos cambios para mejorar el rendimiento<br>
En el futuro verá que diferentes modelos tienen arquitecturas donde puede ajustar los parámetros. Se necesita experimentación para lograr los mejores resultados posibles.